In [1]:
from smml.ofi import divergence
from smml.ofi.divergence import Divergence
from smml.ofi import data
import datetime
import numpy as np
import matplotlib

In [2]:
intc_20120621 = data.lobster.LobsterDataIdentifier(
    ticker='INTC',
    date=datetime.date(2012, 6, 21),
    levels=5,
    file_type=data.lobster.LobsterFileType.ORDERBOOK,
)
aapl_20120621 = data.lobster.LobsterDataIdentifier(
    ticker='AAPL',
    date=datetime.date(2012, 6, 21),
    levels=5,
    file_type=data.lobster.LobsterFileType.ORDERBOOK,
)
msft_20120621 = data.lobster.LobsterDataIdentifier(
    ticker='MSFT',
    date=datetime.date(2012, 6, 21),
    levels=5,
    file_type=data.lobster.LobsterFileType.ORDERBOOK,
)

In [3]:
ldis = [intc_20120621, aapl_20120621, msft_20120621]

In [4]:
t0 = int(60e9)
t1 = int((57600000 - 34200000 - 60)* 1e9)
ewm_decay = .00055
book_levels = 3
include_volume_imbalance = True
sigdegree = 4

In [5]:
bull_samples: data.samples.DivergenceSample = \
    data.samples.bull_only_signature_dataset(
    ldis=ldis,
    t0=t0,
    t1=t1,
    ewm_decay=ewm_decay,
    book_levels=book_levels,
    include_volume_imbalance=include_volume_imbalance,
    sigdegree=sigdegree,
)

In [6]:
bear_samples: data.samples.DivergenceSample = \
    data.samples.bear_only_signature_dataset(
    ldis=ldis,
    t0=t0,
    t1=t1,
    ewm_decay=ewm_decay,
    book_levels=book_levels,
    include_volume_imbalance=include_volume_imbalance,
    sigdegree=sigdegree,
)

In [7]:
bb_samples: data.samples.DivergenceSample = \
    data.samples.bear_bull_signature_dataset(
    ldis=ldis,
    t0=t0,
    t1=t1,
    ewm_decay=ewm_decay,
    book_levels=book_levels,
    include_volume_imbalance=include_volume_imbalance,
    sigdegree=sigdegree,
)

In [8]:
learning_rate = 1e-3
batch_size = 100
shuffle = True
num_of_epochs = 200
verbose = False

In [9]:
bear_div: Divergence = Divergence(
    dataset=bear_samples,
    empirical_sample_size=150,
)

In [10]:
bear_div.train(
    learning_rate=learning_rate,
    batch_size=batch_size,
    shuffle=shuffle,
    num_of_epochs=num_of_epochs,
    verbose=verbose,
)

In [11]:
bear_div_estimate = np.median([bear_div() for _ in range(100)])
print(f'Estimated divergence (bear market): {bear_div_estimate}')

Estimated divergence (bear market): 0.000381476700339849


In [12]:
bull_div: Divergence = Divergence(
    dataset=bull_samples,
    empirical_sample_size=150,
)

In [13]:
bull_div.train(
    learning_rate=learning_rate,
    batch_size=batch_size,
    shuffle=shuffle,
    num_of_epochs=num_of_epochs,
    verbose=verbose,
)

In [14]:
bull_div_estimate = np.median([bull_div() for _ in range(100)])
print(f'Estimated divergence (bull market): {bull_div_estimate}')

Estimated divergence (bull market): 0.00031591338804817445


In [15]:
bb_div: Divergence = Divergence(
    dataset=bb_samples,
    empirical_sample_size=150,
)

In [16]:
bb_div.train(
    learning_rate=learning_rate,
    batch_size=batch_size,
    shuffle=shuffle,
    num_of_epochs=num_of_epochs,
    verbose=verbose,
)

In [17]:
bb_div_estimate = np.median([bb_div() for _ in range(100)])
print(f'Estimated divergence (bear v. bull market): {bb_div_estimate}')

Estimated divergence (bear v. bull market): 0.11039558704678297
